In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
data = open('shakespeare-Copy1.txt','r').read()

In [3]:
data[:50]

'\n                     1\n  From fairest creatures w'

In [4]:
vocab = sorted(set(data))

In [5]:
vocab_size = len(vocab)

In [6]:
vocab_size

84

In [57]:
ind_to_char =np.array( [c for c in vocab])

In [13]:
len(char_to_ind)

84

In [14]:
char_to_ind = {c:i for i,c in enumerate(vocab)}

In [15]:
len(ind_to_char)

84

In [16]:
encoded_text = [char_to_ind[i] for i in data]

In [18]:
len(encoded_text)

5445609

In [19]:
# creating batches

In [22]:
char = tf.data.Dataset.from_tensor_slices(encoded_text)

In [25]:
seq_len =120

In [30]:
sequences=char.batch(seq_len+1,drop_remainder=True)

In [31]:
len(sequences)

45005

In [43]:
for i in char.take(500):
    print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [44]:
def gen_batches(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt,target_txt

In [45]:
batches = sequences.map(gen_batches)

In [65]:
for input_txt,target_txt in batches.take(1):
    print(''.join(ind_to_char[input_txt]))
    print(''.join(ind_to_char[target_txt]))


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [63]:
batch_size = 128

buffer_size = 1000

In [50]:
dataset = batches.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [66]:
# creating model

In [67]:
from tensorflow.keras.models import Sequential

In [68]:
from tensorflow.keras.layers import Dense,LSTM,GRU,Embedding

In [75]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [159]:
def create_model(rnn_size,emb_size,batch_size,vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size,emb_size,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_size,recurrent_initializer='glorot_uniform',return_sequences=True,stateful=True))
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam',loss=sparse_cat_loss)
    return model

In [155]:
rnn_size = 1026

batch_size =128

emb_size = 64

In [156]:
model = create_model(rnn_size,emb_size,batch_size,vocab_size)

In [142]:
print(type(model))

<class 'keras.src.engine.sequential.Sequential'>


In [143]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (128, None, 64)           5376      
                                                                 
 gru_7 (GRU)                 (128, None, 1026)         3361176   
                                                                 
 dense_7 (Dense)             (128, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [144]:
Exam_predictions = []

for input_txt,target_txt in dataset.take(1):
    Exam_predictions = model(input_txt)

In [145]:
Exam_predictions[0]
# probabilities of one-hot-encoded-categories

<tf.Tensor: shape=(120, 84), dtype=float32, numpy=
array([[-1.0492762e-03, -6.6338212e-04, -2.2136548e-04, ...,
         2.8731977e-03, -5.6101642e-05,  4.2138709e-07],
       [ 3.7156946e-03, -1.0655681e-03, -2.7959826e-03, ...,
         5.4829079e-03, -9.9989586e-04,  7.0013851e-03],
       [ 7.3859463e-03,  4.2944504e-03, -3.2772732e-04, ...,
        -7.2629331e-04,  3.4896836e-03, -2.1646140e-04],
       ...,
       [ 1.0316651e-02,  1.3318637e-03,  2.2226917e-03, ...,
         5.0282101e-03,  4.6777092e-03,  6.1680679e-03],
       [ 7.6129229e-04, -1.8166645e-03,  8.4991514e-04, ...,
         3.1477327e-03,  4.0336414e-03,  3.7568123e-03],
       [ 1.0554125e-03,  4.1397605e-03,  1.0418331e-03, ...,
         1.5684376e-03,  4.1810335e-03, -6.0021020e-03]], dtype=float32)>

In [146]:
sample_pred = tf.random.categorical(Exam_predictions[0],num_samples=1)

In [147]:
sample_pred=tf.squeeze(sample_pred,axis=-1)

In [148]:
sample_pred

<tf.Tensor: shape=(120,), dtype=int64, numpy=
array([54,  8, 20, 15, 26,  6, 74, 23, 66, 16, 59,  9, 21, 18, 46, 75,  7,
       35, 74, 33, 43, 34, 10, 45, 24, 42, 77, 61, 59, 32,  0,  2, 62, 68,
       63, 80, 54, 19, 19, 63, 16, 61, 78, 32, 41, 25, 48, 79, 40,  9, 37,
       59, 69, 53, 23, 36, 33, 31, 51, 75, 62, 50, 36,  7, 40, 27,  9, 13,
       70, 53, 58, 62, 74, 49, 74, 74, 77, 31, 31, 44, 42,  9, 49, 64, 79,
       22, 41, 34, 76,  8, 83, 21, 28, 22,  0, 49, 23,  7, 70, 62, 50, 50,
       43, 42, 47, 42, 20, 79, 64, 61, 25, 69, 75,  2, 37, 68, 20, 24, 10,
       80], dtype=int64)>

In [149]:
''.join(ind_to_char[sample_pred])

'_,94A(s<k5d-:7Ut)JsHRI.T>QvfdG\n!gmhy_88h5fwGP?WxO-Ldn]<KHFZtgYK)OB-2o]cgsXssvFFSQ-Xix;PIu,}:C;\nX<)ogYYRQVQ9xif?nt!Lm9>.y'

In [150]:
# generate_text

In [181]:
def generate_text(model,gen_size,start_seed,temp):
    generated_text=[]
    encod = [char_to_ind[c] for c in start_seed]
    encod=tf.expand_dims(encod,0)
    model.reset_states()
    for i in range(gen_size):
        predictions=model(encod)
        
        predictions=tf.squeeze(predictions,0)
        
        predictions = predictions/temp
        
        predictions_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
        
        encod = tf.expand_dims([predicted_id], 0)
        
        generated_text.append(ind_to_char[predictions_id])
        
    return (start_seed + ''.join(text_generated))
        
    

In [191]:
model = create_model(rnn_size,emb_size,batch_size,vocab_size)

In [192]:
model.load_weights('shakespeare_gen-Copy1.h5')

In [193]:
model.build(tf.TensorShape([1,None]))

In [201]:
print(generate_text(model,gen_size=500,start_seed="flower",temp=0.2))

flower all,
    That he should be the company of this company.
    I will not have it so. But I am sorry
    To think that I have seen thee from the way
    To see thee from the princes. Strike on him;
    I have no more mistaken in this place.
    I will not be a sick that have the hand
    Of the rich best advantage of his pride.
    Shall I stand here in this anon?
  TITUS. Why, then we shall have merry be the fire
    That weak and waste of this distill'd grandsires,
    And with the hand of many 


In [200]:
print(generate_text(model,gen_size=500,start_seed="Rama",temp=0.2))

Ramay's
    head. I will not have you heard that which is a gentleman as your books.
    I speak not to the court. For my part, I may say
    'This is not so to me a time to speak.
    There is no trust me with this prison of the place.
    What is the matter, honey nurse, I pray,
    That I might be the sea of many things to see,
    That I might have the charge of this discomfort could
    So much the one as well as I do know.
    There is no time to speak with me to find thee.
  AGUECHEEK. I have


In [205]:
print(generate_text(model,gen_size=5,start_seed="i am",temp=0.2))

i am not 
